In [178]:
import random
import matplotlib
from datetime import datetime
import time
import pandas as pd
import numpy as np
from random import choices
from matplotlib import pyplot as plt
matplotlib.use('TkAgg')  # Use TkAgg instead of InterAgg

# initial conditions
note = "without cache"
population = 60
number_of_generations = 1000
gen_cnt = 0
weight_limit = 800  # Knapsack capacity

knapsack_key = pd.DataFrame([
    [300, 1200],  # High value, moderate weight
    [150, 500],  # Good value-to-weight ratio
    [85, 350],  # Light item, decent value
    [105, 433],  # Moderate weight, solid value
    [30, 192],  # Very light, low value
    [400, 900],  # Heavy but valuable
    [700, 1500],  # Very heavy, but high value
    [250, 750],  # Balanced weight-to-value ratio
    [500, 800],  # Medium-heavy, fair value
    [50, 300]  # Lightweight, reasonable value
], columns=["weight", "value"])


In [179]:

# decorators

def populate(pop_size: int):
    """Decorator iterates the function into a list of the length of the input."""

    def decorator(func):
        def wrapper(*args, **kwargs):
            generation = [func(*args, **kwargs) for _ in range(pop_size)]
            return generation

        return wrapper

    return decorator


def log_time_results(pop:int, gen:int, weight:int, note:str):
    """Decorator that logs the execution time and run time of the function."""

    def decorator(func):
        def wrapper(*args, **kwargs):
            current_date_and_time = datetime.now()
            start = time.time()
            results = func(*args, **kwargs)
            end = time.time()
            total_time = end - start
            with open("log.txt", "a") as file:  # "w" mode creates or overwrites the file
                file.write(
                    f"{current_date_and_time}-: {results} pop:{pop} gens:{gen} weight:{weight} - run time: {total_time:.2f}->{note}\n")
            print(f"Total run time: {total_time}")
            return results

        return wrapper

    return decorator

In [180]:

@populate(population)
def generate_population(input_dataframe:object)->list:
    """takes as input the problem dataframe to be optimized and outputs an initial generation of random genomes.
    Input: pandas dataframe
    function converts length of the dataframe, which are the number of parameters to be solved for, into genomes of the same length. A decorator function will iterate this tasks according to the number of required generations.
    Output: List of lists example: [0, 1, 0, 1]"""
    genome_length = len(input_dataframe)
    return choices([0, 1], k=genome_length)
test_generation = generate_population(knapsack_key)
print(test_generation)

[[1, 1, 0, 1, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 1, 0, 0], [0, 1, 1, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 1, 1, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 1, 0, 1, 1], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0], [0, 1, 1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 1, 0, 0, 1, 1, 0, 1], [1, 0, 0, 0, 1, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 1, 1, 0, 1, 1], [0, 1, 0, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 1, 0, 0, 1, 0, 0, 1], [1, 1, 1, 0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 0, 1, 0, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 0, 1], [0, 0, 0, 1, 0, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 0, 1, 0, 0], [1, 0, 0, 1, 0, 1, 1, 1, 0, 0], [0, 0, 0, 1, 0, 0, 1, 1, 1, 0], [1, 1, 0, 0, 1, 0, 1, 1, 0, 1], [1, 0, 0, 0, 0, 0, 0, 1, 1, 1], [1, 0, 1, 1, 0, 1, 1, 1, 1, 1], [0, 0, 1, 0, 1, 0, 1, 1, 0, 0], [1, 0, 1, 0, 0, 1, 1, 1, 1, 0], [1, 0, 0, 1, 1, 1, 1, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 1, 0, 1, 1, 0, 1, 0], [1, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 

In [181]:
def cache(func):
    def wrapper():
        cache = {}
        

def evaluate_genome(input_dataframe:object, current_generation:list, constraint:int)->object:
    """takes as input the initial dataframe with the problem conditions, the constraint,
    such as the weight limit, and a list of genomes. The function evaluates each gene in the genome and assigns weight
    and value, and then determines if the genome is fit or not based on if it fits the constraint
    :param input_dataframe: 
    :param current_generation: 
    :param constraint: 
    :return: dataframe
    """
    fit_generation = {"genome": [], "weight": [], "value": [], "is fit": []}
    for genome in current_generation:
        genome_logic = np.array(genome).astype(bool)
        weight = input_dataframe[genome_logic]["weight"].sum()
        value = input_dataframe[genome_logic]["value"].sum()
        if weight > constraint or value == 0:
            is_fit = 0
        else:
            is_fit = 1
        fit_generation["genome"].append(genome)
        fit_generation["weight"].append(weight)
        fit_generation["value"].append(value)
        fit_generation["is fit"].append(is_fit)

    return pd.DataFrame(fit_generation)

evaluated_test_generation = evaluate_genome(knapsack_key, test_generation, weight_limit)
print(evaluated_test_generation)

                            genome  weight  value  is fit
0   [1, 1, 0, 1, 1, 0, 1, 1, 1, 1]    2085   5675       0
1   [0, 1, 1, 1, 0, 1, 1, 1, 0, 0]    1690   4433       0
2   [0, 1, 1, 0, 0, 0, 1, 0, 1, 0]    1435   3150       0
3   [0, 1, 0, 0, 1, 0, 0, 0, 0, 0]     180    692       1
4   [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]     750   1550       1
5   [0, 0, 0, 1, 1, 1, 0, 1, 1, 1]    1335   3375       0
6   [0, 1, 1, 0, 1, 0, 1, 0, 1, 1]    1515   3642       0
7   [0, 1, 1, 0, 0, 0, 0, 0, 1, 0]     735   1650       1
8   [0, 1, 1, 1, 0, 1, 1, 1, 1, 0]    2190   5233       0
9   [1, 1, 1, 1, 0, 0, 1, 1, 0, 1]    1640   5033       0
10  [1, 0, 0, 0, 1, 0, 1, 0, 0, 0]    1030   2892       0
11  [1, 0, 1, 0, 0, 1, 1, 0, 1, 1]    2035   5050       0
12  [0, 1, 0, 1, 1, 1, 0, 1, 0, 0]     935   2775       0
13  [1, 1, 1, 1, 0, 0, 1, 0, 0, 1]    1390   4283       0
14  [1, 1, 1, 0, 0, 1, 0, 0, 0, 0]     935   2950       0
15  [1, 1, 1, 0, 0, 0, 0, 1, 1, 0]    1285   3600       0
16  [0, 0, 0, 

In [182]:

def check_fitness(input_dataframe:object, current_generation:list, constraint:int, filtered: bool = True)->object:
    """takes as input the initial dataframe with the problem conditions, the constraint,
    such as the weight limit, and a list of genomes to be evaluated
    :param input_dataframe: 
    :param current_generation: 
    :param constraint: weight limit
    :param filtered: A True boolean removed duplicates and resets the index
    :return: a dataframe with only fit genomes
    """
    filtered_df = evaluate_genome(input_dataframe, current_generation,constraint)
    filtered_df = filtered_df[filtered_df["is fit"] == 1]
    filtered_df = filtered_df.sort_values("value", ascending=False)
    if filtered:
        filtered_df = filtered_df.drop_duplicates("genome")
        filtered_df = filtered_df.reset_index(drop=True)
    # max_value = filtered_df.value.values.max()
    return filtered_df

test_fitness_check = check_fitness(knapsack_key, test_generation, weight_limit)
print(test_fitness_check)

                            genome  weight  value  is fit
0   [1, 0, 0, 0, 1, 1, 0, 0, 0, 1]     780   2592       1
1   [1, 0, 0, 0, 1, 1, 0, 0, 0, 0]     730   2292       1
2   [1, 0, 1, 0, 0, 0, 0, 0, 0, 1]     435   1850       1
3   [0, 1, 1, 0, 0, 1, 0, 0, 0, 0]     635   1750       1
4   [0, 0, 1, 1, 0, 1, 0, 0, 0, 0]     590   1683       1
5   [0, 1, 1, 0, 0, 0, 0, 0, 1, 0]     735   1650       1
6   [0, 0, 0, 1, 0, 1, 0, 0, 0, 1]     555   1633       1
7   [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]     750   1550       1
8   [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]     430   1442       1
9   [0, 0, 1, 1, 1, 0, 0, 0, 0, 1]     270   1275       1
10  [0, 1, 0, 0, 1, 0, 0, 0, 0, 0]     180    692       1


In [183]:
def fitness_function(dataframe:object)->object:
    """Evaluates the current generation as a dataframe, then selects two parents using the roulette wheel
    selection method.
    :param current generation dataframe: 
    :return: parent dataframe: 
    """
    probability = [0] * (len(dataframe))
    probability_total = [0] * (len(dataframe))
    sum_values = dataframe.value.values.sum()
    for i in dataframe.index:
        probability[i] = float(dataframe.value[i] / sum_values)
        probability_total[i] = float(sum(probability))
    r1_select = 1; r2_select = 1
    wheel = [0]
    wheel = wheel + probability_total
    r1 = random.randint(1, 100) / 100
    r2 = random.randint(1, 100) / 100
    for n, i in enumerate(probability_total):
        if wheel[n] < r1 <= probability_total[n]:
            r1_select = n
        if wheel[n] < r2 <= probability_total[n]:
            r2_select = n
    filtered_df = dataframe.iloc[[r1_select, r2_select]]
    parents = filtered_df.sort_values("value", ascending=False).reset_index(drop=True)
    return parents
test_parents = fitness_function(test_fitness_check)
print(test_parents)

                           genome  weight  value  is fit
0  [0, 0, 1, 1, 0, 1, 0, 0, 0, 0]     590   1683       1
1  [0, 0, 1, 1, 1, 0, 0, 0, 0, 1]     270   1275       1


In [185]:

def single_point_crossover(parents_df:object)->tuple:
    """Takes as input a value pair of parents genomes and outputs two new offspring genomes
    :param parents_df: 
    :return: 
    """
    parents = parents_df.genome.tolist()
    genome_1, genome_2 = parents
    if len(genome_1) != len(genome_2):
        raise ValueError("Length of genomes do not match")
    genome_length = len(genome_1)
    slice_location = random.randint(1, genome_length - 1)
    slice_a_1 = genome_1[:slice_location]
    slice_a_2 = genome_1[slice_location:]
    slice_b_1 = genome_2[:slice_location]
    slice_b_2 = genome_2[slice_location:]
    offspring_1 = slice_a_1 + slice_b_2
    offspring_2 = slice_b_1 + slice_a_2
    return offspring_1, offspring_2
test_offspring = single_point_crossover(test_parents)
print(test_offspring)

([0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0])


In [184]:
def mutate(genome:list, probability:int=3, option:str=None)->list:
    """Takes as input one genome as a list and randomly mutates the genome based on the probability input
    :param genome: list i.e. [1, 0, 1, 0, 1, 1, 0, 0, 0, 1]
    :param probability: percentage as an integer
    :param option: optional prompt for debugging
    :return: genome as a list
    """
    genome_length = len(genome)
    mutate_roll = random.randint(1, 100)
    if mutate_roll < probability:
        mutate_index = random.randint(0, genome_length - 1)
        if genome[mutate_index] == 0:
            genome[mutate_index] = 1
            if option is None:
                return genome
            elif option == "declare":
                print("Mutated!")
                return genome
        else:
            genome[mutate_index] = 0
            if option is None:
                return genome
            elif option == "declare":
                print("Mutated!")
                return genome
    else:
        if option is None:
            return genome
        elif option == "declare":
            print("Not mutated")
            return genome 
test_mutate = mutate(test_offspring[0],3,"declare")
print(test_mutate)

Not mutated
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0]


In [186]:
def evolve(parents_df:object, population_size:int)->list:
    """Takes as the input a dataframe containing parent genomes and creates a new generation
    :param parents_df: 
    :param population_size: 
    :return: 
    """
    population_size = int((population_size - 2) / 2) # two parents and the rest offspring
    generation = parents_df.genome.tolist()
    for i in range(population_size):
        off_spring = single_point_crossover(parents_df) # single point crossover function
        generation += [off_spring[0]] + [off_spring[1]]
    return generation
test_evolve = evolve(test_parents,population)
print(test_evolve)
print(len(test_evolve))
print(type(test_evolve))

[[0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 1], [0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 1], [0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 1], [0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1], [0, 0, 

In [187]:

@log_time_results(population, number_of_generations, weight_limit, note)
def genetic_algorithm(input_dataframe:object, population_size:int, generations:int, weight_limit:int, options: str = None)->object:
    """Final implementation of the algorithm incorporating all the functions.
    :param input_dataframe: 
    :param population_size: 
    :param generations: 
    :param weight_limit: 
    :param options: 
    :return: result dataframe
    """
    first_generation = generate_population(input_dataframe)
    current_generation = first_generation 
    solution_vec = []
    for _ in range(generations):
        unfit_removed = check_fitness(knapsack_key, current_generation, weight_limit) #<-------------Check Fitness Function
        parents_df = fitness_function(unfit_removed)  #<-------------------------------Fitness Function selects parents
        current_generation = evolve(parents_df, population_size) #<-------------Evolve Function creates new generation
        solution_row = parents_df.loc[parents_df["value"].idxmax()] #<---------------Stores selected parents
        solution_vec.append(solution_row.genome)
        for n, i in enumerate(current_generation):  
            current_generation[n] = mutate(current_generation[n])  #<-------------Mutate Function iterates across generation 
    # final_generation = check_fitness(knapsack_key, current_generation, weight_limit, filtered=True)
    
    # Compiles results
    solution_df = check_fitness(knapsack_key, solution_vec, weight_limit, filtered=True)
    print(solution_df.sort_values("value", ascending=False))
    solution_vec = solution_df.value.tolist()
    max_solution = solution_df.loc[solution_df["value"].idxmax()]
    weight = max_solution.weight
    max_value = solution_df.value.values.max()
    sol = [max_value.tolist(), weight.tolist()]
    if options == "plot":
        x_vec = range(0, len(solution_vec))
        plt.plot(x_vec, solution_vec)
        return sol, [x_vec, solution_vec]
    return sol


results_ga = genetic_algorithm(knapsack_key, population, number_of_generations, weight_limit,)
print(f"results: {results_ga}")


                            genome  weight  value  is fit
0   [1, 0, 1, 1, 0, 0, 0, 1, 0, 1]     790   3033       1
1   [1, 1, 1, 1, 1, 0, 0, 0, 0, 1]     720   2975       1
2   [1, 1, 0, 0, 1, 0, 0, 1, 0, 1]     780   2942       1
3   [1, 0, 1, 1, 1, 0, 0, 1, 0, 0]     770   2925       1
4   [1, 0, 0, 1, 1, 0, 0, 1, 0, 1]     735   2875       1
..                             ...     ...    ...     ...
77  [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]     305   1233       1
78  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]     300   1200       1
79  [0, 0, 1, 1, 0, 0, 0, 0, 0, 1]     240   1083       1
80  [0, 0, 0, 0, 0, 0, 0, 1, 0, 1]     300   1050       1
81  [0, 1, 1, 0, 1, 0, 0, 0, 0, 0]     265   1042       1

[82 rows x 4 columns]
Total run time: 19.081667184829712
results: [3033, 790]
